In [13]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Cargar los datos
file_path = "datos_proyecto_ML.xlsx"
try:
    df = pd.read_excel(file_path, sheet_name='Hoja1')
except FileNotFoundError:
    print(f"Error: No se encontró el archivo {file_path}. Asegúrate de que esté en la misma carpeta que este notebook.")
    exit()

# Verificar nombres de columnas y eliminar espacios
df.columns = df.columns.str.strip()

# Convertir columnas a numérico asegurando que los valores no sean texto
cols_to_convert = ["EDAD", "PESO (Kg)", "TALLA (mt)", "IMC", "DG NUTRICIONAL", "PAS", "PAD", "GLICEMIA", "COLESTEROL", "CC"]
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Imputar valores faltantes con la mediana
imputer = SimpleImputer(strategy='median')
df[cols_to_convert] = imputer.fit_transform(df[cols_to_convert])

# Crear la variable objetivo "Riesgo_CV" si no existe
df["Riesgo_CV"] = (
    (df["PAS"] >= 140) |  # Hipertensión sistólica
    (df["PAD"] >= 90) |   # Hipertensión diastólica
    (df["COLESTEROL"] >= 200) |  # Colesterol alto
    (df["GLICEMIA"] >= 126) |  # Diabetes
    (df["IMC"] >= 25)  # Sobrepeso u obesidad
).astype(int)

# Seleccionar las variables relevantes
features = ["EDAD", "PESO (Kg)", "TALLA (mt)", "IMC", "DG NUTRICIONAL", "PAS", "PAD", "GLICEMIA", "COLESTEROL", "CC"]
target = "Riesgo_CV"

# Asegurar que hay suficientes datos después de la limpieza
if df.shape[0] == 0:
    print("Error: No quedan datos suficientes después de la limpieza. Revisa el archivo Excel y asegúrate de que contiene datos válidos.")
    exit()

X = df[features]
y = df[target]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelo de Regresión Logística
log_reg = LogisticRegression(class_weight="balanced", max_iter=2000, random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Predicciones
y_pred = log_reg.predict(X_test_scaled)
y_prob = log_reg.predict_proba(X_test_scaled)[:, 1]

# Evaluación
test_accuracy = accuracy_score(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_prob)
report = classification_report(y_test, y_pred)

# Guardar modelo y scaler
joblib.dump(log_reg, "modelo_riesgo_cardiovascular.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(imputer, "imputer.pkl")

# Mostrar resultados
print(f"Precisión: {test_accuracy:.4f}")
print(f"AUC-ROC: {test_roc_auc:.4f}")
print("\nReporte de Clasificación:")
print(report)

Precisión: 0.9648
AUC-ROC: 0.9938

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80        30
           1       1.00      0.96      0.98       396

    accuracy                           0.96       426
   macro avg       0.83      0.98      0.89       426
weighted avg       0.98      0.96      0.97       426



/Users/juanvidalp/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/juanvidalp/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/juanvidalp/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/juanvidalp/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: divide by zero encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/Users/juanvidalp/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_linear_loss.py:330: RuntimeWarning: overflow encountered in matmul
  grad[:n_features] = X.T @ 